In [16]:
# standard library
import os
import pandas as pd
import pandas_gbq as gbq
import numpy as np
import requests
from dotenv import load_dotenv
load_dotenv(dotenv_path="../config/.env")


# datetime and timezone library
from datetime import datetime
from pytz import timezone

# geocoding library
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Google API libraries
from google.oauth2 import service_account

# Logging
import logging
# setup logging
logging.basicConfig(level=logging.INFO)


## Extracting Data from Open Weather Map


In [17]:
# 1. Load .env dan ambil API Key
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="../config/.env")

API_KEY = os.getenv("OWM_API_KEY")
print("API KEY FOUND:", bool(API_KEY))

API KEY FOUND: True


In [18]:
# Define Base URL
API_KEY = os.getenv("OWM_API_KEY")
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
CITY_LIST_PATH = "../config/city_list.csv"

In [19]:
def fetch_weather_by_id(city_id: int) -> dict:
    """Fetch weather data by OpenWeatherMap city ID"""
    params = {
        "id": city_id,
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()
        return {
            "city_id": data["id"],
            "city": data["name"],
            "country": data["sys"]["country"],
            "lat": data["coord"]["lat"],
            "lon": data["coord"]["lon"],
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "weather": data["weather"][0]["main"],
            "description": data["weather"][0]["description"],
            "wind_speed": data["wind"]["speed"],
            "timestamp": datetime.utcfromtimestamp(data["dt"]),
            "fetched_at": datetime.utcnow()
        }
    except Exception as e:
        logging.error(f"Error fetching city ID {city_id}: {e}")
        return None

def fetch_weather_all_cities(city_file: str = CITY_LIST_PATH) -> pd.DataFrame:
    cities_df = pd.read_csv(city_file)
    records = []
    for _, row in cities_df.iterrows():
        result = fetch_weather_by_id(row["id"])
        if result:
            records.append(result)
        else:
            logging.warning(f"Skipping city ID {row['id']} ({row['name']})")
    return pd.DataFrame(records)


In [20]:
data = fetch_weather_all_cities()

C:\Users\LENOVO LOQ\AppData\Local\Temp\ipykernel_15724\662480816.py:23: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "timestamp": datetime.utcfromtimestamp(data["dt"]),
C:\Users\LENOVO LOQ\AppData\Local\Temp\ipykernel_15724\662480816.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "fetched_at": datetime.utcnow()


In [21]:
df = data
df

,city_id,city,country,lat,lon,temperature,humidity,weather,description,wind_speed,timestamp,fetched_at
0,1650077,Banyuwangi,ID,-8.2325,114.3576,26.14,83,Clouds,broken clouds,2.28,2025-07-28 02:10:15,2025-07-28 02:16:49.625100
1,1625822,Surabaya,ID,-7.2492,112.7508,29.82,73,Clouds,few clouds,1.62,2025-07-28 02:06:01,2025-07-28 02:16:49.725272
2,1835848,Seoul,KR,37.5683,126.9778,33.23,62,Clear,clear sky,2.29,2025-07-28 02:08:17,2025-07-28 02:16:49.830537
3,5128581,New York,US,40.7143,-74.0060,25.54,86,Clear,clear sky,2.18,2025-07-28 02:05:42,2025-07-28 02:16:49.937427
4,292223,Dubai,AE,25.2582,55.3047,34.55,51,Clouds,overcast clouds,2.59,2025-07-28 02:06:59,2025-07-28 02:16:50.040582
5,1850147,Tokyo,JP,35.6895,139.6917,33.15,49,Clouds,broken clouds,3.23,2025-07-28 02:07:22,2025-07-28 02:16:50.140919
6,360630,Cairo,EG,30.0626,31.2497,28.44,54,Clear,clear sky,3.30,2025-07-28 02:06:31,2025-07-28 02:16:50.241766
7,1273294,Delhi,IN,28.6667,77.2167,30.37,71,Clouds,overcast clouds,1.83,2025-07-28 02:10:16,2025-07-28 02:16:50.344734
8,3435910,Buenos Aires,AR,-34.6132,-58.3772,9.93,79,Rain,light rain,7.52,2025-07-28 02:10:16,2025-07-28 02:16:50.446986
9,1816670,Beijing,CN,39.9075,116.3972,26.91,88,Clouds,overcast clouds,0.84,2025-07-28 02:10:07,2025-07-28 02:16:50.550718


In [22]:
df.describe()

,city_id,lat,lon,temperature,humidity,wind_speed,timestamp,fetched_at
count,3.400000e+01,34.000000,34.000000,34.000000,34.000000,34.000000,34,34
mean,2.154912e+06,16.801215,54.171185,23.712941,70.735294,3.257059,2025-07-28 02:09:02.382352640,2025-07-28 02:16:51.315735296
min,2.922230e+05,-34.613200,-79.416300,5.640000,49.000000,0.710000,2025-07-28 02:04:24,2025-07-28 02:16:49.625100
25%,1.266719e+06,-5.114475,15.421400,17.747500,62.000000,1.672500,2025-07-28 02:07:04.500000,2025-07-28 02:16:50.472919040
50%,1.826259e+06,28.639750,66.260700,25.200000,71.500000,2.295000,2025-07-28 02:08:39,2025-07-28 02:16:51.319389440
75%,2.887547e+06,40.512600,115.887300,29.767500,83.000000,4.110000,2025-07-28 02:10:17,2025-07-28 02:16:52.160100608
max,6.167865e+06,55.752200,153.028100,34.550000,89.000000,10.210000,2025-07-28 02:14:22,2025-07-28 02:16:53.000003
std,1.488930e+06,29.323654,71.656588,7.571767,12.903626,2.368454,NaN,NaN


# Descriptive Analysis - OpenWeatherMap ETL Pipeline

This document summarizes descriptive insights gathered from a single ETL pipeline execution fetching real-time weather data from 50 global cities using the OpenWeatherMap API.

## Purpose of ETL Pipeline

The pipeline is designed to automate the collection of real-time weather data from multiple global cities, clean and enrich it, store it in Google BigQuery, and present it through an interactive dashboard. This supports analysis, monitoring, and visualization of global weather conditions on a recurring basis.

---

## Descriptive Insights

### 1. Distribution of Weather Conditions

- Most cities during the snapshot were experiencing **Clouds** or **Clear** skies.
- A few cities were reporting **Rain** or **Moderate Rain** (e.g., Addis Ababa, Manila).

**Interpretation:** Globally, the dominant real-time weather pattern was overcast or clear, with localized rainfall events.

---

### 2. Temperature Extremes

- **Highest Temperatures:**
  - Cairo (40.75°C)
  - Dubai (41.97°C)
  - Delhi (37.6°C)

- **Lowest Temperatures:**
  - Cape Town (16.23°C)
  - Sydney (16.03°C)
  - São Paulo (15.99°C)

**Interpretation:** Cities in arid or subtropical zones recorded significantly higher temperatures, supporting potential alert classification.

---

### 3. Temperature Categories

(Assuming enrichment step added a `temp_category` column.)

- **Hot:** Cairo, Dubai, Delhi
- **Warm:** Jakarta, Tokyo, New York
- **Cold:** Cape Town, São Paulo, Sydney

**Interpretation:** This classification allows for geographical grouping and temporal comparison across cities.

---

### 4. Notable Wind Conditions

- Cities such as **Osaka**, **Bengaluru**, and **Istanbul** recorded wind speeds above 8 m/s.

**Interpretation:** These cities may be experiencing wind-driven weather activity, which is relevant for alerts and infrastructure planning.

---

### 5. Combined Conditions (Risk Zones)

- **Cairo** and **Dubai**: High temperatures with clear skies (potential heatwave zones)
- **Manila**: High temperature + rain (high humidity zone)

---

## Why These Insights Matter

These insights justify the creation of the ETL pipeline:

- **Without automation**, gathering this scale of data daily would be unfeasible.
- **Enables trend detection**, anomaly identification, and support for weather-sensitive decisions (e.g., agriculture, disaster readiness, transportation).
- **Scalable foundation** for adding alerting, forecasting, or geospatial mapping in the future.

---

## Conclusion

The descriptive analysis demonstrates that even a single ETL run can offer a valuable snapshot of global weather variation. With continuous automation, the data becomes increasingly powerful for monitoring trends and supporting decision-making.

## Transforming and Enriching the Data

In [23]:
# Default: Waktu Jakarta (bisa disesuaikan)
WIB = timezone("Asia/Jakarta")

def clean_weather_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Bersihkan dan transformasi data cuaca:
    - Pastikan tipe data konsisten
    - Drop null penting
    - Konversi timestamp ke timezone lokal
    """
    if df.empty:
        return df

    df = df.dropna(subset=["temperature", "humidity", "weather", "timestamp"])

    df["timestamp"] = pd.to_datetime(df["timestamp"]).dt.tz_localize("UTC").dt.tz_convert(WIB)
    df["fetched_at"] = pd.to_datetime(df["fetched_at"]).dt.tz_localize("UTC").dt.tz_convert(WIB)

    return df


def enrich_weather_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tambahkan kolom turunan:
    - temp_category (dingin, hangat, panas)
    - local_time, day_of_week, hour_of_day
    """
    if df.empty:
        return df

    def classify_temp(temp):
        if temp < 20:
            return "Cold"
        elif temp < 30:
            return "Warm"
        else:
            return "Hot"

    df["temp_category"] = df["temperature"].apply(classify_temp)
    df["day_of_week"] = df["timestamp"].dt.day_name()
    df["hour_of_day"] = df["timestamp"].dt.hour

    return df

clean = clean_weather_data(df)
enriched = enrich_weather_data(clean)

In [24]:
enriched.head()

,city_id,city,country,lat,lon,temperature,humidity,weather,description,wind_speed,timestamp,fetched_at,temp_category,day_of_week,hour_of_day
0,1650077,Banyuwangi,ID,-8.2325,114.3576,26.14,83,Clouds,broken clouds,2.28,2025-07-28 09:10:15+07:00,2025-07-28 09:16:49.625100+07:00,Warm,Monday,9
1,1625822,Surabaya,ID,-7.2492,112.7508,29.82,73,Clouds,few clouds,1.62,2025-07-28 09:06:01+07:00,2025-07-28 09:16:49.725272+07:00,Warm,Monday,9
2,1835848,Seoul,KR,37.5683,126.9778,33.23,62,Clear,clear sky,2.29,2025-07-28 09:08:17+07:00,2025-07-28 09:16:49.830537+07:00,Hot,Monday,9
3,5128581,New York,US,40.7143,-74.0060,25.54,86,Clear,clear sky,2.18,2025-07-28 09:05:42+07:00,2025-07-28 09:16:49.937427+07:00,Warm,Monday,9
4,292223,Dubai,AE,25.2582,55.3047,34.55,51,Clouds,overcast clouds,2.59,2025-07-28 09:06:59+07:00,2025-07-28 09:16:50.040582+07:00,Hot,Monday,9


In [25]:
enriched.describe()  

,city_id,lat,lon,temperature,humidity,wind_speed,hour_of_day
count,3.400000e+01,34.000000,34.000000,34.000000,34.000000,34.000000,34.0
mean,2.154912e+06,16.801215,54.171185,23.712941,70.735294,3.257059,9.0
std,1.488930e+06,29.323654,71.656588,7.571767,12.903626,2.368454,0.0
min,2.922230e+05,-34.613200,-79.416300,5.640000,49.000000,0.710000,9.0
25%,1.266719e+06,-5.114475,15.421400,17.747500,62.000000,1.672500,9.0
50%,1.826259e+06,28.639750,66.260700,25.200000,71.500000,2.295000,9.0
75%,2.887547e+06,40.512600,115.887300,29.767500,83.000000,4.110000,9.0
max,6.167865e+06,55.752200,153.028100,34.550000,89.000000,10.210000,9.0


# Additional Descriptive Insights — Enriched Weather Data (Sample of 5 Cities)

## 1. Distribution of Temperature Categories

- **Hot:** Surabaya, Seoul, Dubai  
- **Warm:** Banyuwangi, New York

**Interpretation:**  
Despite all cities being observed at the same hour (`hour_of_day = 18`), thermal conditions vary significantly. This highlights the importance of location-based temperature classification for global comparative analysis.

---

## 2. Humidity Extremes

- **Banyuwangi** recorded the highest humidity at **95%**
- **Dubai**, despite having the highest temperature (41.97°C), had the lowest humidity at **35%**

**Interpretation:**  
Such variations are relevant for calculating heat index or thermal discomfort zones. This can be a valuable enrichment for future transformations.

---

## 3. Weather Conditions at the Same Hour

All records are from **18:00 local time**, yet:
- Most cities had **cloud-related conditions** (`Clouds`, `scattered clouds`)
- **Seoul** had a clear sky

**Interpretation:**  
This supports time-based weather pattern analysis and can be used for simple hourly trend prediction models.

---

## 4. Wind Speed Variation

- **Dubai** had the highest wind speed at **7.12 m/s**
- **Banyuwangi** and **Seoul** had lower wind speeds below 2 m/s

**Interpretation:**  
Wind speed variation contributes to weather severity classification or comfort index analysis.

---

## 5. Temporal Consistency

- All data points were fetched on **Sunday** at **hour 18**
- This confirms temporal alignment of the dataset, which is crucial for time-series analysis and consistent dashboard visuals

---

## Recommendations

- Derive a `heat_index` field from temperature and humidity
- Analyze `temp_category` trends across hours to identify daily thermal cycles
- Create scatter plots of `humidity vs temperature` to detect thermal outliers


In [26]:
df.to_csv(f'../data/weather-daily-{datetime.now().strftime("%Y-%m-%d")}.csv', index=False)

In [28]:
# BigQuery Configuration
PROJECT_ID = os.getenv("PROJECT_ID")
TABLE_ID = os.getenv("TABLE_ID")
CREDENTIALS_PATH = "../config/owm-etl-project-986252b4609c.json"

In [29]:
from pandas_gbq import to_gbq
def upload_to_bigquery(
    df: pd.DataFrame,
    project_id: str = PROJECT_ID,
    table_id: str = TABLE_ID,
    credentials_path: str = CREDENTIALS_PATH,
    if_exists: str = "replace"
):
    """
    Unggah DataFrame ke Google BigQuery.

    Parameters:
        df (pd.DataFrame): Data yang akan diunggah
        project_id (str): ID project GCP
        table_id (str): Format dataset.table
        credentials_path (str): Path ke JSON file service account
        if_exists (str): Mode upload: 'replace', 'append', 'fail'
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(credentials_path)
        logging.info(f"Uploading to BigQuery: {table_id} (mode: {if_exists})")
        to_gbq(
            dataframe=df,
            destination_table=table_id,
            project_id=project_id,
            credentials=credentials,
            if_exists=if_exists
        )
        logging.info("✅ Upload success.")
    except Exception as e:
        logging.error(f"❌ Upload failed: {e}")

In [30]:
upload_to_bigquery(enriched, if_exists="append")

INFO:root:Uploading to BigQuery: weather_data.daily (mode: append)
34 out of 34 rows loaded.ctor:
INFO:root:✅ Upload success.
